In [8]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [4]:
adeft_grounder = AdeftGrounder()

In [5]:
shortforms = ['ARG']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [6]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [7]:
list(zip(longforms, counts))

[('arginine', 28),
 ('arctigenin', 10),
 ('arginase', 5),
 ('aggressive response generation', 3),
 ('amphiregulin', 3)]

In [13]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-27 03:12:13] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [14]:
result = [grounding_map, names, pos_labels]

In [15]:
result

[{'aggressive response generation': 'ungrounded',
  'amphiregulin': 'HGNC:651',
  'arctigenin': 'CHEBI:CHEBI:79',
  'arginase': 'MESH:D001119',
  'arginine': 'CHEBI:CHEBI:29016'},
 {'HGNC:651': 'AREG',
  'CHEBI:CHEBI:79': '(-)-Arctigenin',
  'MESH:D001119': 'Arginase',
  'CHEBI:CHEBI:29016': 'arginine'},
 ['CHEBI:CHEBI:29016', 'CHEBI:CHEBI:79', 'HGNC:651', 'MESH:D001119']]

In [80]:
grounding_map, names, pos_labels = [{'aggressive response generation': 'ungrounded',
  'amphiregulin': 'HGNC:651',
  'arctigenin': 'CHEBI:CHEBI:79',
  'arginase': 'MESH:D001119',
  'arginine': 'CHEBI:CHEBI:29016'},
 {'HGNC:651': 'AREG',
  'CHEBI:CHEBI:79': '(-)-Arctigenin',
  'MESH:D001119': 'Arginase',
  'CHEBI:CHEBI:29016': 'arginine'},
 ['CHEBI:CHEBI:29016', 'CHEBI:CHEBI:79', 'HGNC:651', 'MESH:D001119']]

In [81]:
excluded_longforms = []

In [82]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [83]:
additional_entities = {}

In [84]:
unambiguous_agent_texts = {}

In [85]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [86]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [87]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [88]:
intersection1

[]

In [89]:
intersection2

[]

In [78]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [79]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [90]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-27 03:25:38] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-27 03:25:39] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.82 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [91]:
classifier.stats

{'label_distribution': {'HGNC:651': 3,
  'CHEBI:CHEBI:29016': 18,
  'MESH:D001119': 4,
  'CHEBI:CHEBI:79': 4,
  'ungrounded': 1},
 'f1': {'mean': 0.82, 'std': 0.09798},
 'precision': {'mean': 0.794667, 'std': 0.11964},
 'recall': {'mean': 0.86, 'std': 0.071181},
 'HGNC:651': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'CHEBI:CHEBI:29016': {'f1': {'mean': 0.977778, 'std': 0.044444},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.96, 'std': 0.08}},
 'CHEBI:CHEBI:79': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D001119': {'f1': {'mean': 0.466667, 'std': 0.4},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.374166}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}}}

In [92]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [93]:
disamb.dump(model_name, results_path)

In [94]:
print(disamb.info())

Disambiguation model for ARG

Produces the disambiguations:
	(-)-Arctigenin*	CHEBI:CHEBI:79
	AREG*	HGNC:651
	Arginase*	MESH:D001119
	arginine*	CHEBI:CHEBI:29016

Class level metrics:
--------------------
Grounding     	Count	F1     
      arginine*	18	0.97778
      Arginase*	 4	0.46667
(-)-Arctigenin*	 4	    0.4
          AREG*	 3	    0.4
    Ungrounded	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.82
	Precision:	0.79467
	Recall:		0.86

* Positive labels
See Docstring for explanation



In [95]:
model_to_s3(disamb)